## This demo app shows:
* How to run Llama2 locally on a Mac using llama-cpp-python and the llama-cpp's quantized Llama2 model
* How to use LangChain to ask Llama general questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

We start by installing necessary requirements and import packages we will be using in this example.
- [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) a simple Python bindings for [llama.cpp](https://github.com/ggerganov/llama.cpp) library
- pypdf gives us the ability to work with pdfs
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

In [2]:
pip install cmake


  Using cached cmake-3.27.7-py2.py3-none-win_amd64.whl.metadata (6.8 kB)
Using cached cmake-3.27.7-py2.py3-none-win_amd64.whl (34.6 MB)


In [7]:
pip install llama-cpp-python

     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.8 MB 445.2 kB/s eta 0:00:18
     ---------------------------------------- 0.0/7.8 MB 281.8 kB/s eta 0:00:28
     ---------------------------------------- 0.1/7.8 MB 438.1 kB/s eta 0:00:18
      --------------------------------------- 0.1/7.8 MB 610.6 kB/s eta 0:00:13
     - -------------------------------------- 0.2/7.8 MB 734.2 kB/s eta 0:00:11
     - -------------------------------------- 0.3/7.8 MB 984.6 kB/s eta 0:00:08
     -- ------------------------------------- 0.5/7.8 MB 1.2 MB/s eta 0:00:06
     -- ------------------------------------- 0.5/7.8 MB 1.3 MB/s eta 0:00:06
     --- ------------------------------------ 0.8/7.8 MB 1.7 MB/s eta 0:00:05
     ----- ---------------------------------- 1.0/7.8 MB 2.0 MB/s eta 0:00:04
     ------ --------------------------------- 1.2/7.8 MB 2.1 MB/s 

In [1]:
# install all the required packages for the demo
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install pypdf sentence-transformers chromadb langchain

'CMAKE_ARGS' is not recognized as an internal or external command,
operable program or batch file.


     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ------------- ------------------------ 30.7/86.0 kB 145.2 kB/s eta 0:00:01
     ------------------ ------------------- 41.0/86.0 kB 140.3 kB/s eta 0:00:01
     --------------------------- ---------- 61.4/86.0 kB 204.8 kB/s eta 0:00:01
     ------------------------------------ - 81.9/86.0 kB 241.3 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 241.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/123.1 kB ? eta -:--:--
     ------------------------- ------------- 81.9/123.1 kB 4.8

In [1]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [2]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model. 

Replace `<path-to-llama-gguf-file>` with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing), 

or to the `ggml-model-q4_0.gguf` file built with the following commands:

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0

```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [3]:

llm = LlamaCpp(
    model_path=r"C:\Users\razit\Downloads\llama-2-7b-chat.Q4_0.gguf",
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager, 
    verbose=True,
)

AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 0 | SSSE3 = 0 | VSX = 0 | 


With the model set up, you are now ready to ask some questions. 

Here is an example of the simplest way to ask the model some general questions.

In [4]:

question = "I want something to drink water in it. sumerize your answer and only give me only some keywords to search in google?"
answer = llm(question)


 Unterscheidung between the two words "water" and "drink" is important because they have different meanings and connotations. For example, you might search for "water bottle" if you're looking for a container to hold water, but you might search for "drink recipes" if you're looking for ideas for beverages to consume.
Here are some possible search queries related to the topic of drinking water:
1. Water bottles - This search query could lead you to websites selling reusable water bottles or other containers for carrying water on the go.
2. Drink recipes - If you're looking for ideas for beverages to consume, this search query could lead you to websites with recipes for homemade drinks like juice, smoothies, and more.
3. Water purification systems - If you're concerned about the quality of your tap water or want to have a reliable source of clean drinking water when traveling, this search query could lead you to information on water filtration systems or other purification methods.
4. H

Alternatively, you can use LangChain's `PromptTemplate` for some flexibility in your prompts and questions.

For more information on LangChain's prompt template visit this [link](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

In [ ]:
# a more flexible way to ask Llama general questions using LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    "who wrote {book}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("innovator's dilemma")

Now, let's see how Llama2 hallucinates, because it did not have knowledge about Llama2 at the time it was trained. 
By default it behaves like a know-it-all expert who will not say "I don't know".

In [ ]:

prompt = PromptTemplate.from_template(
    "What is {what}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("llama2")

One way we can fix the hallucinations is to use RAG, to augment it with more recent or custom data that holds the information for it to answer correctly.

First we load the Llama2 paper using LangChain's [PDF loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [ ]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("llama2.pdf")
documents = loader.load()

In [ ]:
# quick check on the loaded document for the correct pages etc
print(len(documents), documents[0].page_content[0:300])

Next we will store our documents. 
There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the `HuggingFaceEmbeddings` and `RecursiveCharacterTextSplitter` to assist in storing the documents.

In [ ]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 


In [ ]:
# split the loaded documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)


We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

In [ ]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)



For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to the model to answer the question.

It takes close to 2 minutes to return the result (but using other vector stores other than Chroma such as FAISS can take longer) because Llama2 is running on a local Mac. 
To get much faster results, you can use a cloud service with GPU used for inference - see HelloLlamaCloud for a demo.

In [ ]:

question = "What is llama2?"
result = qa_chain({"query": question})
